# Gather Hourly Traffic Data <br>
*From statens vegvesen API: https://www.vegvesen.no/trafikkdata/api*

Import necessary packages:

In [1]:
import os
import json
import requests
import time
import random
from tqdm import tqdm  
import pandas as pd
from datetime import datetime
# Display all columns in the df
pd.set_option('display.max_columns', None)

Set paths etc.:

In [2]:
# define the base path where the file should be saved
base_path = '/Users/filipmoennilsen/Documents/Skolearbeid/BI_Master_BAN/4. Semester/Master_Thesis/Master_Python/roadNetwork_Venv/PythonFiles/trafficAPI/2_TrafficOutputData'
# define the file name
file_name = "all_IDs.json"
# combine the base path and file name to create a valid file path
file_path = os.path.join(base_path, file_name)

### First query (get traffic point ID's):

In [3]:
# define the endpoint of the API
endpoint = 'https://www.vegvesen.no/trafikkdata/api/'

# define the GraphQL query for fetching ID's of registration points (only Europe roads)
query = """
{
  trafficRegistrationPoints(searchQuery: {roadCategoryIds: [E ,R, F], registrationFrequency: CONTINUOUS}) {
    id
  }
}
"""

# make a POST request to the API
response = requests.post(endpoint, json={"query": query})

# check the status code of the response
if response.status_code == 200:
    # if the request was successful, get the data from the response
    data = response.json()

    # open the file for writing
    with open(file_path, "w") as file:
        # write the data to the file in JSON format
        json.dump(data, file)

Store query number 1 and convert ID's to list for inserting in query number 2:

In [4]:
# define the file path of the JSON file
file_path = '/Users/filipmoennilsen/Documents/Skolearbeid/BI_Master_BAN/4. Semester/Master_Thesis/Master_Python/roadNetwork_Venv/PythonFiles/trafficAPI/2_TrafficOutputData/all_IDs.json'

# open the file for reading
with open(file_path, "r") as file:
    # load the data from the file into a Python object
    data = json.load(file)

# create an empty list for the registration point ID's
registration_point_ids = []

# iterate over the data returned by the GraphQL query
for item in data["data"]["trafficRegistrationPoints"]:
    # get the ID of the current registration point
    id = item["id"]

    # add the ID to the list of registration point ID's
    registration_point_ids.append(id)

### Second query (get traffic volume for every hour in a one-week period): <br>
*NB! This query must be run **twice** because it only possible to get 100 hours per query.* <br>
*The dataframes must therefore be merged together later on.*

In [5]:
# define the endpoint of the API
endpoint = 'https://www.vegvesen.no/trafikkdata/api/'

data_1 = []
with tqdm(total=len(registration_point_ids)) as pbar:     # HUSK Å FJERNE [:300] FOR Å FÅ MED ALLE TRAFFIC POINTS
  for registration_point_id in registration_point_ids:    # HUSK Å FJERNE [:300] FOR Å FÅ MED ALLE TRAFFIC POINTS
    time.sleep(random.uniform(1, 3))

    # Change to: "2018-09-23T23:00:00+02:00"
    query = """ {
  trafficData(trafficRegistrationPointId: "%s") {
    volume {
      byHour(
        from: "2018-07-16T00:00:00+02:00"
        to: "2018-07-19T11:00:00+02:00"  
      ) {
        edges {
          node {
            from
            to
            total {
              volumeNumbers {
                volume
              }
              coverage {
                percentage
              }
            }
          }
        }
      }
    }
  }
}
    """ % registration_point_id

    response = requests.post(endpoint, json={"query": query})

    if response.status_code == 200:
      volume_data_1 = response.json()

      # Add the registration_point_id to the volume_data dictionary
      volume_data_1["registration_point_id"] = registration_point_id

      data_1.append(volume_data_1)

      pbar.update(1)

100%|██████████| 3126/3126 [1:47:52<00:00,  2.07s/it]  


*Second run:*

In [6]:
# define the endpoint of the API
endpoint = 'https://www.vegvesen.no/trafikkdata/api/'

data_2 = []
with tqdm(total=len(registration_point_ids)) as pbar:     # HUSK Å FJERNE [:300] FOR Å FÅ MED ALLE TRAFFIC POINTS
  for registration_point_id in registration_point_ids:    # HUSK Å FJERNE [:300] FOR Å FÅ MED ALLE TRAFFIC POINTS
    time.sleep(random.uniform(1, 3))

    # Change to: "2018-09-23T23:00:00+02:00"
    query = """ {
  trafficData(trafficRegistrationPointId: "%s") {
    volume {
      byHour(
        from: "2018-07-19T11:00:00+02:00"
        to: "2018-07-23T00:00:00+02:00"
      ) {
        edges {
          node {
            from
            to
            total {
              volumeNumbers {
                volume
              }
              coverage {
                percentage
              }
            }
          }
        }
      }
    }
  }
}
    """ % registration_point_id

    response = requests.post(endpoint, json={"query": query})

    if response.status_code == 200:
      volume_data_2 = response.json()

      # Add the registration_point_id to the volume_data dictionary
      volume_data_2["registration_point_id"] = registration_point_id

      data_2.append(volume_data_2)

      pbar.update(1)

100%|██████████| 3126/3126 [1:47:43<00:00,  2.07s/it]  


Store data collected from query number 2:

In [7]:
# define the path and filename for the JSON file
filename_1 = '/Users/filipmoennilsen/Documents/Skolearbeid/BI_Master_BAN/4. Semester/Master_Thesis/Master_Python/roadNetwork_Venv/PythonFiles/trafficAPI/2_TrafficOutputData/hourlyVolume_1_Jul.json'
# write the data to the JSON file
with open(filename_1, "w") as f:
    json.dump(data_1, f, indent=2) # usually indent = 4, but ok 

# define the path and filename for the JSON file
filename_2 = '/Users/filipmoennilsen/Documents/Skolearbeid/BI_Master_BAN/4. Semester/Master_Thesis/Master_Python/roadNetwork_Venv/PythonFiles/trafficAPI/2_TrafficOutputData/hourlyVolume_2_Jul.json'
# write the data to the JSON file
with open(filename_2, "w") as f:
    json.dump(data_2, f, indent=2) 

Open data collected from query number 2:

In [8]:
with open('/Users/filipmoennilsen/Documents/Skolearbeid/BI_Master_BAN/4. Semester/Master_Thesis/Master_Python/roadNetwork_Venv/PythonFiles/trafficAPI/2_TrafficOutputData/hourlyVolume_1_Jul.json', 'r') as file:
    json_data_1 = json.load(file)

with open('/Users/filipmoennilsen/Documents/Skolearbeid/BI_Master_BAN/4. Semester/Master_Thesis/Master_Python/roadNetwork_Venv/PythonFiles/trafficAPI/2_TrafficOutputData/hourlyVolume_2_Jul.json', 'r') as file:
    json_data_2 = json.load(file)

### def flatten_json <br>
*Takes the data gathered from query 2 (stored as a json-file), flattening the data and converts it to a pandas dataframe.*

In [9]:
def flatten_json(json_data):
    flat_list = []
    for item in json_data:
        edges = item.get("data", {}).get("trafficData", {}).get("volume", {}).get("byHour", {}).get("edges", [])
        traffic_registration_point_id = item.get("registration_point_id")

        data_count = 0
        temp_list = []
        for edge in edges:
            node = edge.get("node", {})
            total = node.get("total", {})

            if total is None:
                continue

            volume_numbers = total.get("volumeNumbers") or {}
            coverage = total.get("coverage") or {}

            if volume_numbers.get("volume") is None:
                continue

            from_datetime = datetime.fromisoformat(node.get("from").replace("+02:00", ""))
            to_datetime = datetime.fromisoformat(node.get("to").replace("+02:00", ""))

            flat_dict = {
                "traffic_point_id": traffic_registration_point_id,
                "year": from_datetime.year,
                "month": from_datetime.month,
                "day": from_datetime.day,
                "from_hour": from_datetime.strftime("%H:%M"),
                "to_hour": to_datetime.strftime("%H:%M"),
                "volume": volume_numbers.get("volume"),
                "coverage_percentage": coverage.get("percentage")
            }
            data_count += 1
            temp_list.append(flat_dict)

        if data_count >= 83:    # 83 er riktig tall egentlig            # NB! Denne burde egentlig være "if data_count == 168:", men det ser ut til at det kun er mulig å hente ut 100 timer på ett punkt om gangen.
                                            # data_count == 168 fordi 24 timer i 7 dager (24*7) = 168
            flat_list.extend(temp_list)

    return flat_list


**Read warning in function above.**

Run function, check output:

In [10]:
flat_data_1 = flatten_json(json_data_1)
flat_data_2 = flatten_json(json_data_2)
# Convert flattened data to DataFrame
volume_df_1 = pd.DataFrame(flat_data_1)
volume_df_2 = pd.DataFrame(flat_data_2)
display(volume_df_1)#[:50]
display(volume_df_2)#[:50]

,traffic_point_id,year,month,day,from_hour,to_hour,volume,coverage_percentage
0,65823V1668921,2018,7,16,00:00,01:00,29,100.0
1,65823V1668921,2018,7,16,01:00,02:00,3,100.0
2,65823V1668921,2018,7,16,02:00,03:00,6,100.0
3,65823V1668921,2018,7,16,03:00,04:00,17,100.0
4,65823V1668921,2018,7,16,04:00,05:00,4,100.0
...,...,...,...,...,...,...,...,...
156284,10795V320297,2018,7,19,06:00,07:00,207,100.0
156285,10795V320297,2018,7,19,07:00,08:00,339,100.0
156286,10795V320297,2018,7,19,08:00,09:00,290,100.0
156287,10795V320297,2018,7,19,09:00,10:00,352,100.0


,traffic_point_id,year,month,day,from_hour,to_hour,volume,coverage_percentage
0,65823V1668921,2018,7,19,11:00,12:00,237,100.0
1,65823V1668921,2018,7,19,12:00,13:00,275,100.0
2,65823V1668921,2018,7,19,13:00,14:00,271,100.0
3,65823V1668921,2018,7,19,14:00,15:00,260,100.0
4,65823V1668921,2018,7,19,15:00,16:00,234,100.0
...,...,...,...,...,...,...,...,...
162234,10795V320297,2018,7,22,19:00,20:00,364,100.0
162235,10795V320297,2018,7,22,20:00,21:00,371,100.0
162236,10795V320297,2018,7,22,21:00,22:00,230,100.0
162237,10795V320297,2018,7,22,22:00,23:00,186,100.0


### Merge the two dataframes:

In [11]:
# Concatenate the two dataframes
merged_df = pd.concat([volume_df_1, volume_df_2], ignore_index=True)

# Group by traffic_point_id and filter based on the number of data points (168 hours)
completeVolume_df = merged_df.groupby('traffic_point_id').filter(lambda x: len(x) >= 166)

# Reset the index of the filtered dataframe
completeVolume_df.reset_index(drop=True, inplace=True)


Check the merged dataframe - looks good:

In [12]:
completeVolume_df#[:60]

,traffic_point_id,year,month,day,from_hour,to_hour,volume,coverage_percentage
0,65823V1668921,2018,7,16,00:00,01:00,29,100.0
1,65823V1668921,2018,7,16,01:00,02:00,3,100.0
2,65823V1668921,2018,7,16,02:00,03:00,6,100.0
3,65823V1668921,2018,7,16,03:00,04:00,17,100.0
4,65823V1668921,2018,7,16,04:00,05:00,4,100.0
...,...,...,...,...,...,...,...,...
313125,10795V320297,2018,7,22,19:00,20:00,364,100.0
313126,10795V320297,2018,7,22,20:00,21:00,371,100.0
313127,10795V320297,2018,7,22,21:00,22:00,230,100.0
313128,10795V320297,2018,7,22,22:00,23:00,186,100.0


Save dataframe as CSV-file in directory:

In [13]:
# Specify the file path and name
output_file_path = "/Users/filipmoennilsen/Documents/Skolearbeid/BI_Master_BAN/4. Semester/Master_Thesis/Master_Python/roadNetwork_Venv/PythonFiles/trafficAPI/2_TrafficOutputData/completeAllHourlyVolume_Jul.csv"

# Convert the dataframe to a CSV file and save it to the specified path
completeVolume_df.to_csv(output_file_path, index=False)


In [14]:
print(completeVolume_df['traffic_point_id'].nunique() / len(registration_point_ids))

0.5962891874600128


In [15]:
#completeVolume_df = pd.read_csv('/Users/filipmoennilsen/Documents/Skolearbeid/BI_Master_BAN/4. Semester/Master_Thesis/Master_Python/roadNetwork_Venv/PythonFiles/trafficAPI/2_TrafficOutputData/completeAllHourlyVolume_July.csv')
#completeVolume_df

### Pivot the dataframe <br> 
*This is to display hourly volume on the y-axis (not sure if necessary)*

In [16]:
# Create a new column 'timestamp' combining year, month, day, and from_hour
completeVolume_df['timestamp'] = completeVolume_df['year'].astype(str) + '-' + completeVolume_df['month'].astype(str).str.zfill(2) + '-' + completeVolume_df['day'].astype(str).str.zfill(2) + ' ' + completeVolume_df['from_hour']

# Pivot the DataFrame using 'traffic_point_id' as index and 'timestamp' as columns
pivoted_df = completeVolume_df.pivot_table(index='traffic_point_id', columns='timestamp', values='volume')

# Reset index and fill missing values with 0 (if any)
pivoted_df.reset_index(inplace=True)
pivoted_df.fillna(0, inplace=True)

# Display pivoted dataframe
pivoted_df

timestamp,traffic_point_id,2018-07-16 00:00,2018-07-16 01:00,2018-07-16 02:00,2018-07-16 03:00,2018-07-16 04:00,2018-07-16 05:00,2018-07-16 06:00,2018-07-16 07:00,2018-07-16 08:00,2018-07-16 09:00,2018-07-16 10:00,2018-07-16 11:00,2018-07-16 12:00,2018-07-16 13:00,2018-07-16 14:00,2018-07-16 15:00,2018-07-16 16:00,2018-07-16 17:00,2018-07-16 18:00,2018-07-16 19:00,2018-07-16 20:00,2018-07-16 21:00,2018-07-16 22:00,2018-07-16 23:00,2018-07-17 00:00,2018-07-17 01:00,2018-07-17 02:00,2018-07-17 03:00,2018-07-17 04:00,2018-07-17 05:00,2018-07-17 06:00,2018-07-17 07:00,2018-07-17 08:00,2018-07-17 09:00,2018-07-17 10:00,2018-07-17 11:00,2018-07-17 12:00,2018-07-17 13:00,2018-07-17 14:00,2018-07-17 15:00,2018-07-17 16:00,2018-07-17 17:00,2018-07-17 18:00,2018-07-17 19:00,2018-07-17 20:00,2018-07-17 21:00,2018-07-17 22:00,2018-07-17 23:00,2018-07-18 00:00,2018-07-18 01:00,2018-07-18 02:00,2018-07-18 03:00,2018-07-18 04:00,2018-07-18 05:00,2018-07-18 06:00,2018-07-18 07:00,2018-07-18 08:00,2018-07-18 09:00,2018-07-18 10:00,2018-07-18 11:00,2018-07-18 12:00,2018-07-18 13:00,2018-07-18 14:00,2018-07-18 15:00,2018-07-18 16:00,2018-07-18 17:00,2018-07-18 18:00,2018-07-18 19:00,2018-07-18 20:00,2018-07-18 21:00,2018-07-18 22:00,2018-07-18 23:00,2018-07-19 00:00,2018-07-19 01:00,2018-07-19 02:00,2018-07-19 03:00,2018-07-19 04:00,2018-07-19 05:00,2018-07-19 06:00,2018-07-19 07:00,2018-07-19 08:00,2018-07-19 09:00,2018-07-19 10:00,2018-07-19 11:00,2018-07-19 12:00,2018-07-19 13:00,2018-07-19 14:00,2018-07-19 15:00,2018-07-19 16:00,2018-07-19 17:00,2018-07-19 18:00,2018-07-19 19:00,2018-07-19 20:00,2018-07-19 21:00,2018-07-19 22:00,2018-07-19 23:00,2018-07-20 00:00,2018-07-20 01:00,2018-07-20 02:00,2018-07-20 03:00,2018-07-20 04:00,2018-07-20 05:00,2018-07-20 06:00,2018-07-20 07:00,2018-07-20 08:00,2018-07-20 09:00,2018-07-20 10:00,2018-07-20 11:00,2018-07-20 12:00,2018-07-20 13:00,2018-07-20 14:00,2018-07-20 15:00,2018-07-20 16:00,2018-07-20 17:00,2018-07-20 18:00,2018-07-20 19:00,2018-07-20 20:00,2018-07-20 21:00,2018-07-20 22:00,2018-07-20 23:00,2018-07-21 00:00,2018-07-21 01:00,2018-07-21 02:00,2018-07-21 03:00,2018-07-21 04:00,2018-07-21 05:00,2018-07-21 06:00,2018-07-21 07:00,2018-07-21 08:00,2018-07-21 09:00,2018-07-21 10:00,2018-07-21 11:00,2018-07-21 12:00,2018-07-21 13:00,2018-07-21 14:00,2018-07-21 15:00,2018-07-21 16:00,2018-07-21 17:00,2018-07-21 18:00,2018-07-21 19:00,2018-07-21 20:00,2018-07-21 21:00,2018-07-21 22:00,2018-07-21 23:00,2018-07-22 00:00,2018-07-22 01:00,2018-07-22 02:00,2018-07-22 03:00,2018-07-22 04:00,2018-07-22 05:00,2018-07-22 06:00,2018-07-22 07:00,2018-07-22 08:00,2018-07-22 09:00,2018-07-22 10:00,2018-07-22 11:00,2018-07-22 12:00,2018-07-22 13:00,2018-07-22 14:00,2018-07-22 15:00,2018-07-22 16:00,2018-07-22 17:00,2018-07-22 18:00,2018-07-22 19:00,2018-07-22 20:00,2018-07-22 21:00,2018-07-22 22:00,2018-07-22 23:00
0,00000V1702725,13.0,3.0,0.0,0.0,2.0,20.0,90.0,248.0,248.0,504.0,922.0,1134.0,1119.0,1182.0,1211.0,1231.0,1173.0,948.0,915.0,797.0,668.0,334.0,70.0,28.0,13.0,7.0,1.0,3.0,4.0,18.0,89.0,218.0,230.0,430.0,1016.0,1065.0,1103.0,1114.0,1133.0,1210.0,1116.0,892.0,869.0,732.0,612.0,304.0,51.0,32.0,18.0,6.0,2.0,1.0,3.0,21.0,88.0,210.0,229.0,368.0,702.0,971.0,953.0,958.0,960.0,1070.0,1049.0,887.0,890.0,760.0,562.0,350.0,69.0,31.0,28.0,8.0,4.0,1.0,2.0,16.0,80.0,214.0,183.0,384.0,690.0,910.0,957.0,1013.0,1029.0,1107.0,1104.0,902.0,822.0,730.0,581.0,313.0,65.0,27.0,15.0,9.0,1.0,3.0,2.0,15.0,75.0,206.0,193.0,367.0,685.0,976.0,991.0,1079.0,1071.0,1205.0,1113.0,962.0,818.0,709.0,565.0,323.0,72.0,40.0,26.0,13.0,9.0,2.0,0.0,8.0,20.0,37.0,89.0,300.0,660.0,992.0,1157.0,1225.0,1229.0,1120.0,1077.0,1021.0,727.0,459.0,128.0,62.0,47.0,38.0,30.0,17.0,6.0,4.0,3.0,1.0,2.0,13.0,6.0,32.0,67.0,80.0,128.0,137.0,139.0,156.0,164.0,186.0,125.0,139.0,105.0,79.0,39.0,22.0
1,00000V1702751,471.0,125.0,96.0,77.0,147.0,538.0,1024.0,1577.0,1514.0,2544.0,1559.0,3810.0,3803.0,3722.0,3681.0,3992.0,3722.0,3268.0,3012.0,2597.0,1895.0,1350.0,832.

Store new pivoted dataframe:

In [17]:
# Specify the file path and name
output_file_path = "/Users/filipmoennilsen/Documents/Skolearbeid/BI_Master_BAN/4. Semester/Master_Thesis/Master_Python/roadNetwork_Venv/PythonFiles/trafficAPI/2_TrafficOutputData/completeAllHourlyVolume_Jul_pivoted.csv"

# Convert the dataframe to a CSV file and save it to the specified path
pivoted_df.to_csv(output_file_path, index=False)